# Smart Beta Portfolio Optimization with cvxpy and S&P500

The objective is to design a portfolio that closely tracks an index, while also minimizing the portfolio variance.  If this portfolio can match the returns of the index with less volatility, then it has a higher risk-adjusted return (same return, lower volatility).

We want to both minimize the portfolio variance and also want to closely track a market cap weighted index.  In other words, we're trying to minimize the distance between the weights of our portfolio and the weights of the index.

$Minimize \left [ \sigma^2_p + \lambda \sqrt{\sum_{1}^{m}(weight_i - indexWeight_i)^2} \right  ]$ where $m$ is the number of stocks in the portfolio, and $\lambda$ is a scaling factor that you can choose.

Why are we doing this? One way that investors evaluate a fund is by how well it tracks its index. The fund is still expected to deviate from the index within a certain range in order to improve fund performance.  A way for a fund to track the performance of its benchmark is by keeping its asset weights similar to the weights of the index.  We’d expect that if the fund has the same stocks as the benchmark, and also the same weights for each stock as the benchmark, the fund would yield about the same returns as the benchmark. By minimizing a linear combination of both the portfolio risk and distance between portfolio and benchmark weights, we attempt to balance the desire to minimize portfolio variance with the goal of tracking the index.



## Import libraries

In [1]:
pip install yfinance

     |████████████████████████████████| 6.3 MB 2.3 MB/s 
  Created wheel for yfinance: filename=yfinance-0.1.63-py2.py3-none-any.whl size=23918 sha256=3482fe247f3e08f7746f985b0d708e61125356c8a449948264c38111f16d900e
  Stored in directory: /root/.cache/pip/wheels/fe/87/8b/7ec24486e001d3926537f5f7801f57a74d181be25b11157983
Successfully built yfinance
  Attempting uninstall: lxml
    Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6


In [2]:
import numpy as np
import pandas as pd
import cvxpy as cvx
import yfinance as yf
from scipy import stats
import requests
import bs4 as bs
import matplotlib.pyplot as plt
plt.style.use("seaborn-whitegrid")

## Load stock data

Focus on large dollar volume stocks to ensure liquidity.

In [3]:
# get S&P500 tickers from Wikipedia
resp = requests.get('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
soup = bs.BeautifulSoup(resp.text, 'lxml')
table = soup.find('table', {'id': 'constituents'})
tickers = []

for row in table.find_all('tr')[1:]:
		ticker = row.find('td').text.strip()
		if "." in ticker:
			ticker = ticker.replace('.','-')
		tickers.append(ticker)

In [4]:
df = yf.download(tickers=tickers, period="max", interval="1d", group_by="ticker", auto_adjust=True)
df

[*********************100%***********************]  505 of 505 completed


MCO                          ...     HAL                        
                  Open        High         Low  ...     Low      Close       Volume
Date                                            ...                                
1962-01-02         NaN         NaN         NaN  ...     NaN        NaN          NaN
1962-01-03         NaN         NaN         NaN  ...     NaN        NaN          NaN
1962-01-04         NaN         NaN         NaN  ...     NaN        NaN          NaN
1962-01-05         NaN         NaN         NaN  ...     NaN        NaN          NaN
1962-01-08         NaN         NaN         NaN  ...     NaN        NaN          NaN
...                ...         ...         ...  ...     ...        ...          ...
2021-08-09  385.859985  387.529999  381.510010  ...  19.260  19.690001  757870000.0
2021-08-10  383.160004  383.980011  378.209991  ...  19.730  20.209999  688870000.0
2021-08-11  382.209991  382.500000  378.730011  ...  19.850  20.629999  592690000.0
2021-08-12  378.769989  380.100006  376.279999  ...     NaN        NaN          NaN
2021-08-13  380.250000  381.850006  379.510010  ...  20.045  20.129999    6133550.0

[15021 rows x 2525 columns]

## Get close prices

In [5]:
sp_close = df.copy()
sp_close = sp_close.loc[:, sp_close.columns.get_level_values(1)=="Close"].fillna(method="ffill") # ffill to fill any missing values (hilidays) with previous day's value
sp_close.columns = sp_close.columns.get_level_values(0)
sp_close = sp_close.loc["2017-01-01":"2021-01-01", :].dropna(axis=1)
sp_close

,MCO,UDR,INFO,PSX,LEN,MKC,CBOE,ROP,ECL,CCL,XEL,EBAY,ZBH,AMGN,DIS,NWL,HD,PCAR,CZR,TGT,SJM,BKNG,TWTR,BKR,DOV,GPN,IPG,ABBV,NCLH,CFG,AZO,AAL,NOC,PRU,ROL,TDY,VIAC,REG,CPB,ISRG,...,PSA,STZ,DLR,FCX,AVB,CTXS,TSN,DRE,AVY,ES,MXIM,STX,PWR,RMD,CBRE,ABMD,JCI,CERN,BR,AXP,WYNN,KSU,AKAM,COP,FMC,DUK,SYF,AMT,PFG,ORLY,BWA,O,AJG,MGM,GOOGL,BRK-B,SWK,PENN,PBCT,HAL
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2017-01-02,90.062187,31.098055,34.887691,72.775047,41.094288,43.372322,72.269569,178.083954,111.877106,46.718204,35.701473,28.655676,99.670624,128.616455,99.803192,36.998692,120.499199,54.374939,16.950001,63.264210,111.560402,1466.060059,16.299999,39.456711,55.380695,68.828239,19.513409,50.367065,42.529999,30.206615,789.789978,45.118092,216.506760,85.604767,14.225926,123.000000,58.393520,57.831268,52.182659,211.389999,...,189.597748,143.505173,84.260338,12.728151,152.075638,68.738747,55.834671,22.792631,64.560509,48.345905,34.312244,30.227861,34.443512,58.546768,31.490000,112.680000,36.625893,46.231110,61.457729,68.948296,80.152527,80.456879,66.680000,44.199352,45.904251,63.429226,32.243515,96.771965,48.487022,278.410004,36.684875,47.046230,47.009689,27.245058,792.450012,162.979996,105.761497,12.827439,15.675654,49.141174
2017-01-03,91.208656,30.646250,35.390163,73.095078,41.084713,43.493149,71.408875,179.883438,112.659721,46.789986,35.631283,28.800444,99.796181,132.592545,101.584358,37.222424,120.705864,55.583279,17.049999,63.719666,112.353157,1477.569946,16.440001,39.493141,56.481964,69.909111,19.596766,50.198158,42.790001,30.401608,791.460022,44.741226,218.657150,86.262878,14.103798,124.000000,59.100262,57.780941,52.329361,213.929993,...,189.614716,144.853043,85.203636,13.297493,151.620667,69.369865,56.649380,22.844116,64.486946,48.214603,34.356724,31.376163,34.502808,58.452408,31.570000,112.360001,37.728489,48.212307,61.114761,70.130295,81.032707,76.635529,67.570000,44.613667,46.285702,63.519115,32.945805,97.202339,49.090385,280.820007,37.270866,47.054394,47.263020,27.859324,808.010010,163.830002,106.738983,13.115802,15.618979,50.585697
2017-01-04,92.756317,30.953140,35.666039,73.490929,42.099400,43.367668,74.215927,180.233643,113.375542,47.400204,35.789192,28.723234,100.713699,134.475021,102.886719,38.539948,121.775360,56.349125,16.900000,64.648087,111.969833,1487.290039,16.860001,39.323105,56.748039,72.517059,19.696793,50.905972,44.770000,30.664413,793.039978,45.127758,218.992264,87.315872,14.230139,124.070000,60.339352,58.703560,52.691799,209.830002,...,191.005936,147.885849,87.904823,14.310727,152.247375,69.477623,57.455044,23.359011,66.325760,48.380917,34.721466,30.980196,34.700474,58.829826,32.080002,115.739998,37.781849,46.777653,61.522602,71.279961,83.645470,76.976883,68.330002,45.027988,47.243382,63.453724,33.719234,97.376328,49.534531,282.109985,37.968479,47.758289,47.860130,27.840420,807.770020,164.080002,107.412155,13.636712,15.869980,50.776489
2017-01-05,93.491959,31.055433,35.754707,73.052963,42.520584,42.517227,73.443260,180.555237,113.203751,48.198895,35.789192,28.964525,101.360779,134.571793,102.829262,38.680820,120.337425,55.430107,16.650000,63.553268,112.536079,1503.699951,17.090000,39.080177,56.563263,72.804626,19.255009,51.292057,44.549999,30.282919,792.000000,44.345032,219.681137,86.007866,14.158546,123.360001,60.100719,58.837757,52.579624,210.970001,...,193.041901,137.364716,90.245857,14.108080,153.174469,69.454536,57.753761,23.264614,65.672974,48.389664,34.196594,30.924770,34.463272,58.376930,31.809999,114.809998,37.292786,46.504387,61.207455,70.401352,84.720230,77.479454,69.550003,44.895756,47.421936,63.535461,32.812462,97.037521,49.031723,282.809998,37.921967,49.043301,47.968708,27.755369,813.020020,163.300003,109.136566,13.376257,15.610883,51.067211
2017-01-06,93.912331,31.308964,36.119251,71.924423,41.821800,42.475418,73.844261,183.089081,113.499611,48.145050,35.894444,29.968290,101.370445,137.914536,104.361458,38.829983,120.004906,56.059803,16.600000,62.572273,112.414131,

## Get volume

In [6]:
sp_volume = df.copy()
sp_volume = sp_volume.loc[:, sp_volume.columns.get_level_values(1)=="Volume"].fillna(method="ffill") # ffill to fill any missing values (hilidays) with previous day's value
sp_volume.columns = sp_volume.columns.get_level_values(0)
sp_volume = sp_volume.loc["2017-01-01":"2021-01-01", :].dropna(axis=1)
sp_volume

,MCO,UDR,INFO,PSX,LEN,MKC,CBOE,ROP,ECL,CCL,XEL,EBAY,ZBH,AMGN,DIS,NWL,HD,PCAR,CZR,TGT,SJM,BKNG,TWTR,BKR,DOV,GPN,IPG,ABBV,NCLH,CFG,AZO,AAL,NOC,PRU,ROL,TDY,VIAC,REG,CPB,ISRG,...,PSA,STZ,DLR,FCX,AVB,CTXS,TSN,DRE,AVY,ES,MXIM,STX,PWR,RMD,CBRE,ABMD,JCI,CERN,BR,AXP,WYNN,KSU,AKAM,COP,FMC,DUK,SYF,AMT,PFG,ORLY,BWA,O,AJG,MGM,GOOGL,BRK-B,SWK,PENN,PBCT,HAL
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2017-01-02,519500.0,1685400.0,2636800.0,1374800.0,2408256.0,1144400.0,522700.0,372300.0,715200.0,2323800.0,1893100.0,8189700.0,973800.0,3193100.0,6983300.0,2071400.0,4149100.0,2122900.0,535300.0,3069200.0,586600.0,407400.0,14014500.0,1872700.0,647226.0,941100.0,1989300.0,5999200.0,856900.0,3394300.0,246400.0,4495000.0,740900.0,1466100.0,1057050.0,209600.0,3811600.0,786100.0,1204500.0,801900.0,...,891500.0,1145200.0,1305800.0,16761800.0,756300.0,845900.0,1792600.0,2503200.0,585400.0,1446800.0,1658300.0,2897500.0,1695800.0,532600.0,1312600.0,166100.0,4885600.0,1780200.0,354500.0,3378800.0,1888500.0,937300.0,967700.0,4806300.0,686400.0,2465600.0,3452600.0,1635500.0,806400.0,578000.0,1326300.0,2024000.0,1221400.0,5461200.0,1735900.0,4041400.0,594300.0,858300.0,2896000.0,4655700.0
2017-01-03,1180900.0,1863000.0,2792200.0,2257900.0,4148852.0,1026800.0,652300.0,421400.0,1080600.0,3450800.0,2065100.0,7664800.0,1381100.0,3640900.0,8884300.0,3827600.0,5299700.0,2115600.0,490200.0,3907100.0,652500.0,458500.0,11147900.0,2710300.0,1637998.0,1640300.0,2266100.0,9328200.0,1618500.0,5045900.0,248800.0,6737800.0,1178900.0,2128300.0,1115775.0,128600.0,3553800.0,1012000.0,1185400.0,792900.0,...,938100.0,1991400.0,1412600.0,28586200.0,858200.0,1314000.0,4199000.0,2135600.0,865600.0,1123100.0,1614300.0,5129300.0,1311200.0,573900.0,2282800.0,284400.0,9453800.0,2681100.0,498700.0,5853900.0,2703500.0,2920500.0,1077400.0,7032600.0,795500.0,2905900.0,6058100.0,2240900.0,1275600.0,967500.0,1485500.0,1973300.0,1058600.0,9439200.0,1959000.0,4091000.0,731600.0,1071200.0,3897600.0,10531700.0
2017-01-04,859900.0,1815400.0,1651300.0,2306900.0,2653760.0,1335800.0,871300.0,436900.0,720000.0,3169800.0,2542600.0,9538800.0,1502800.0,3945900.0,8322000.0,5471200.0,4055300.0,2448400.0,425800.0,5316200.0,786900.0,387700.0,15861300.0,2122600.0,1165948.0,1769700.0,3127600.0,8674800.0,1854500.0,4354900.0,187200.0,5859600.0,664100.0,2071700.0,1201275.0,216800.0,5365500.0,718900.0,1257500.0,1361700.0,...,723300.0,2972300.0,2490700.0,36310900.0,759700.0,1418900.0,2615300.0,2806900.0,887700.0,1264600.0,1892300.0,3842700.0,1184000.0,758800.0,2145100.0,335800.0,5106800.0,3869500.0,630800.0,4635800.0,2598800.0,4636700.0,1174300.0,4990000.0,804900.0,2531100.0,6040100.0,1720700.0,995200.0,492600.0,1600900.0,1808600.0,1382800.0,6330900.0,1515300.0,3568900.0,940100.0,1337600.0,3477500.0,6265900.0
2017-01-05,1274500.0,2907400.0,1786700.0,2056300.0,1969319.0,2789800.0,591500.0,342700.0,731500.0,4022300.0,2611900.0,9062200.0,1453900.0,3441700.0,6303000.0,3151100.0,3821100.0,1658300.0,218100.0,7096300.0,661300.0,447500.0,17074400.0,2028500.0,1155178.0,1741400.0,5353900.0,5359400.0,933700.0,3725200.0,243200.0,6825300.0,891200.0,1720400.0,2436525.0,124200.0,4186500.0,629300.0,1488500.0,1110600.0,...,956600.0,10511800.0,2819800.0,34956000.0,840600.0,841300.0,2474400.0,2213700.0,647200.0,1590700.0,2142100.0,2889500.0,1375400.0,627000.0,1824500.0,383000.0,5184800.0,2909900.0,735300.0,3383000.0,2507500.0,2346900.0,1779800.0,4727100.0,916500.0,3284600.0,6850800.0,1919900.0,1308300.0,644000.0,1775400.0,2276500.0,1309500.0,8587100.0,1340500.0,2982500.0,1944400.0,2506400.0,3229500.0,7083300.0
2017-01-06,1065100.0,1415400.0,1541400.0,1989400.0,2088715.0,2560200.0,732400.0,647800.0,864600.0,3669600.0,1707400.0,13351400.0,1821000.0,9899300.0,9551300.0,3605100.0,3827500.0,1741100.0,219600.0,5235100.0,534600.0,473400.0,14665100.0,2554500.0,3123350.0,1581600.0,3959200.0,4722300.0,932900.0,3623500.0,216500.0,7260200.0,619200.0,1983700.0,927450.0,91800.0,5275500.0,582900.0,888500.

## Index weights

S&P500 is weighted by dividing the free-float market capitalization of a company in the index by the total market capitalization of the index.

In [7]:
def generate_dollar_volume_weights(close, volume):
  """
    Generate dollar volume weights.

    Parameters
    ----------
    close : DataFrame
        Close price for each ticker and date
    volume : str
        Volume for each ticker and date

    Returns
    -------
    dollar_volume_weights : DataFrame
        The dollar volume weights for each ticker and date
  """

  assert close.index.equals(volume.index)
  assert close.columns.equals(volume.columns)
  
  dollar_volume_traded = close * volume
  dollar_volume_weights = dollar_volume_traded.divide(dollar_volume_traded.sum(axis=1), axis=0)

  return dollar_volume_weights

In [8]:
sp_weights = generate_dollar_volume_weights(sp_close, sp_volume)
sp_weights

,MCO,UDR,INFO,PSX,LEN,MKC,CBOE,ROP,ECL,CCL,XEL,EBAY,ZBH,AMGN,DIS,NWL,HD,PCAR,CZR,TGT,SJM,BKNG,TWTR,BKR,DOV,GPN,IPG,ABBV,NCLH,CFG,AZO,AAL,NOC,PRU,ROL,TDY,VIAC,REG,CPB,ISRG,...,PSA,STZ,DLR,FCX,AVB,CTXS,TSN,DRE,AVY,ES,MXIM,STX,PWR,RMD,CBRE,ABMD,JCI,CERN,BR,AXP,WYNN,KSU,AKAM,COP,FMC,DUK,SYF,AMT,PFG,ORLY,BWA,O,AJG,MGM,GOOGL,BRK-B,SWK,PENN,PBCT,HAL
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2017-01-02,0.000555,0.000621,0.001090,0.001186,0.001173,0.000588,0.000448,0.000786,0.000948,0.001287,0.000801,0.002781,0.001150,0.004867,0.008260,0.000908,0.005925,0.001368,0.000108,0.002301,0.000776,0.007079,0.002707,0.000876,0.000425,0.000768,0.000460,0.003581,0.000432,0.001215,0.002306,0.002404,0.001901,0.001487,0.000178,0.000306,0.002638,0.000539,0.000745,0.002009,...,0.002003,0.001948,0.001304,0.002529,0.001363,0.000689,0.001186,0.000676,0.000448,0.000829,0.000674,0.001038,0.000692,0.000370,0.000490,0.000222,0.002121,0.000975,0.000258,0.002761,0.001794,0.000894,0.000765,0.002518,0.000373,0.001854,0.001319,0.001876,0.000463,0.001907,0.000577,0.001129,0.000680,0.001763,0.016303,0.007806,0.000745,0.000130,0.000538,0.002712
2017-01-03,0.000948,0.000503,0.000870,0.001453,0.001501,0.000393,0.000410,0.000667,0.001072,0.001421,0.000648,0.001943,0.001213,0.004250,0.007945,0.001254,0.005631,0.001035,0.000074,0.002192,0.000645,0.005964,0.001613,0.000942,0.000814,0.001009,0.000391,0.004122,0.000610,0.001350,0.001733,0.002654,0.002269,0.001616,0.000139,0.000140,0.001849,0.000515,0.000546,0.001493,...,0.001566,0.002539,0.001060,0.003346,0.001145,0.000802,0.002094,0.000429,0.000491,0.000477,0.000488,0.001417,0.000398,0.000295,0.000634,0.000281,0.003140,0.001138,0.000268,0.003614,0.001929,0.001970,0.000641,0.002762,0.000324,0.001625,0.001757,0.001918,0.000551,0.002392,0.000487,0.000817,0.000440,0.002315,0.013935,0.005900,0.000687,0.000124,0.000536,0.004690
2017-01-04,0.000739,0.000521,0.000546,0.001571,0.001035,0.000537,0.000599,0.000730,0.000756,0.001392,0.000843,0.002539,0.001403,0.004917,0.007935,0.001954,0.004577,0.001279,0.000067,0.003185,0.000817,0.005344,0.002478,0.000774,0.000613,0.001189,0.000571,0.004092,0.000769,0.001238,0.001376,0.002451,0.001348,0.001676,0.000158,0.000249,0.003000,0.000391,0.000614,0.002648,...,0.001280,0.004074,0.002029,0.004816,0.001072,0.000914,0.001393,0.000608,0.000546,0.000567,0.000609,0.001103,0.000381,0.000414,0.000638,0.000360,0.001788,0.001677,0.000360,0.003062,0.002015,0.003308,0.000744,0.002082,0.000352,0.001488,0.001887,0.001553,0.000457,0.001288,0.000563,0.000800,0.000613,0.001633,0.011343,0.005427,0.000936,0.000169,0.000511,0.002948
2017-01-05,0.001105,0.000838,0.000593,0.001394,0.000777,0.001100,0.000403,0.000574,0.000768,0.001799,0.000867,0.002435,0.001367,0.004297,0.006013,0.001131,0.004266,0.000853,0.000034,0.004184,0.000690,0.006242,0.002707,0.000735,0.000606,0.001176,0.000956,0.002550,0.000386,0.001047,0.001787,0.002808,0.001816,0.001373,0.000320,0.000142,0.002334,0.000343,0.000726,0.002174,...,0.001713,0.013395,0.002361,0.004575,0.001194,0.000542,0.001326,0.000478,0.000394,0.000714,0.000680,0.000829,0.000440,0.000340,0.000538,0.000408,0.001794,0.001255,0.000418,0.002209,0.001971,0.001687,0.001148,0.001969,0.000403,0.001936,0.002085,0.001728,0.000595,0.001690,0.000625,0.001036,0.000583,0.002211,0.010110,0.004518,0.001969,0.000311,0.000468,0.003356
2017-01-06,0.000953,0.000422,0.000531,0.001364,0.000832,0.001036,0.000515,0.001130,0.000935,0.001684,0.000584,0.003813,0.001759,0.013011,0.009499,0.001334,0.004377,0.000930,0.000035,0.003122,0.000573,0.006860,0.002400,0.000946,0.001710,0.001113,0.000743,0.002309,0.000393,0.001051,0.001636,0.003090,0.001300,0.001635,0.000124,0.000107,0.002967,0.000328,0.000444,0.001785,...,0.001287,0.005921,0.001073,0.003350,0.001079,0.000842,0.001489,0.000525,0.000456,0.000564,0.000801,0.001316,0.000466,0.000269,0.000611,0.000296,0.002179,0.001625,0.000281,0.002077,0.001517,0.001347,0.001308,0.001480,0.000348,0.001890,0.001607,0.001833,0.

In [9]:
sp_weights.iloc[-1].nlargest(10)

TSLA     0.198737
AAPL     0.074252
AMZN     0.054632
MSFT     0.026300
FB       0.019988
NFLX     0.016539
NVDA     0.014229
AMD      0.012969
BA       0.012734
GOOGL    0.010467
Name: 2020-12-31 00:00:00, dtype: float64

## Returns

In [10]:
def generate_returns(prices):
  """
    Generate returns for ticker and date.

    Parameters
    ----------
    prices : DataFrame
        Price for each ticker and date

    Returns
    -------
    returns : Dataframe
        The returns for each ticker and date
  """

  returns = prices / prices.shift(1) - 1
  return returns

In [11]:
sp_price_returns = generate_returns(sp_close)
sp_price_returns

,MCO,UDR,INFO,PSX,LEN,MKC,CBOE,ROP,ECL,CCL,XEL,EBAY,ZBH,AMGN,DIS,NWL,HD,PCAR,CZR,TGT,SJM,BKNG,TWTR,BKR,DOV,GPN,IPG,ABBV,NCLH,CFG,AZO,AAL,NOC,PRU,ROL,TDY,VIAC,REG,CPB,ISRG,...,PSA,STZ,DLR,FCX,AVB,CTXS,TSN,DRE,AVY,ES,MXIM,STX,PWR,RMD,CBRE,ABMD,JCI,CERN,BR,AXP,WYNN,KSU,AKAM,COP,FMC,DUK,SYF,AMT,PFG,ORLY,BWA,O,AJG,MGM,GOOGL,BRK-B,SWK,PENN,PBCT,HAL
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2017-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-01-03,0.012730,-0.014528,0.014403,0.004398,-0.000233,0.002786,-0.011910,0.010105,0.006995,0.001536,-0.001966,0.005052,0.001260,0.030914,0.017847,0.006047,0.001715,0.022222,0.005900,0.007199,0.007106,0.007851,0.008589,0.000923,0.019885,0.015704,0.004272,-0.003354,0.006113,0.006455,0.002115,-0.008353,0.009932,0.007688,-0.008585,0.008130,0.012103,-0.000870,0.002811,0.012016,...,0.000089,0.009392,0.011195,0.044731,-0.002992,0.009181,0.014591,0.002259,-0.001139,-0.002716,0.001296,0.037988,0.001722,-0.001612,0.002540,-0.002840,0.030104,0.042854,-0.005581,0.017143,0.010981,-0.047496,0.013347,0.009374,0.008310,0.001417,0.021781,0.004447,0.012444,0.008656,0.015974,0.000174,0.005389,0.022546,0.019635,0.005215,0.009242,0.022480,-0.003615,0.029395
2017-01-04,0.016968,0.010014,0.007795,0.005416,0.024697,-0.002885,0.039310,0.001947,0.006354,0.013042,0.004432,-0.002681,0.009194,0.014197,0.012820,0.035396,0.008860,0.013778,-0.008798,0.014570,-0.003412,0.006578,0.025547,-0.004305,0.004711,0.037305,0.005104,0.014100,0.046272,0.008644,0.001996,0.008639,0.001533,0.012207,0.008958,0.000565,0.020966,0.015968,0.006926,-0.019165,...,0.007337,0.020937,0.031703,0.076197,0.004133,0.001553,0.014222,0.022539,0.028515,0.003449,0.010616,-0.012620,0.005729,0.006457,0.016155,0.030082,0.001414,-0.029757,0.006673,0.016393,0.032243,0.004454,0.011248,0.009287,0.020691,-0.001029,0.023476,0.001790,0.009047,0.004594,0.018717,0.014959,0.012634,-0.000679,-0.000297,0.001526,0.006307,0.039716,0.016070,0.003772
2017-01-05,0.007931,0.003305,0.002486,-0.005959,0.010005,-0.019610,-0.010411,0.001784,-0.001515,0.016850,0.000000,0.008401,0.006425,0.000720,-0.000558,0.003655,-0.011808,-0.016309,-0.014793,-0.016935,0.005057,0.011033,0.013642,-0.006178,-0.003256,0.003966,-0.022429,0.007584,-0.004914,-0.012441,-0.001311,-0.017345,0.003146,-0.014980,-0.005031,-0.005723,-0.003955,0.002286,-0.002129,0.005433,...,0.010659,-0.071144,0.026631,-0.014161,0.006089,-0.000332,0.005199,-0.004041,-0.009842,0.000181,-0.015117,-0.001789,-0.006836,-0.007698,-0.008417,-0.008035,-0.012944,-0.005842,-0.005122,-0.012326,0.012849,0.006529,0.017855,-0.002937,0.003779,0.001288,-0.026892,-0.003479,-0.010151,0.002481,-0.001225,0.026907,0.002269,-0.003055,0.006499,-0.004754,0.016054,-0.019100,-0.016326,0.005726
2017-01-06,0.004496,0.008164,0.010196,-0.015448,-0.016434,-0.000983,0.005460,0.014034,0.002614,-0.001117,0.002941,0.034655,0.000095,0.024840,0.014900,0.003856,-0.002763,0.011360,-0.003003,-0.015436,-0.001084,0.011219,0.004681,-0.006061,0.015549,0.014029,0.023377,0.000313,-0.006734,0.005319,0.001149,0.006973,0.003136,0.005739,-0.005354,-0.006080,-0.018173,0.002138,-0.003775,0.005056,...,0.003472,0.018330,-0.011403,0.019152,0.008799,0.008533,-0.007523,-0.002951,0.013580,-0.000362,0.033819,-0.014341,0.000861,0.004364,-0.019176,0.005313,0.020267,-0.002099,0.004089,0.001992,0.010827,0.025701,0.003451,-0.003731,-0.001882,-0.003344,0.005961,-0.006606,0.005982,-0.003642,0.008094,-0.001335,0.002263,0.005107,0.014993,0.000674,-0.001183,-0.044506,0.007261,0.008006
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,.

## Weighted returns

In [12]:
def generate_weighted_returns(returns, weights):
  """
    Generate weighted returns.

    Parameters
    ----------
    returns : DataFrame
        Returns for each ticker and date
    weights : DataFrame
        Weights for each ticker and date

    Returns
    -------
    weighted_returns : DataFrame
        Weighted returns for each ticker and date
  """

  assert returns.index.equals(weights.index)
  assert returns.columns.equals(weights.columns)

  weighted_returns = returns * weights

  return weighted_returns

In [13]:
sp_weighted_returns = generate_weighted_returns(sp_price_returns, sp_weights)
sp_weighted_returns

,MCO,UDR,INFO,PSX,LEN,MKC,CBOE,ROP,ECL,CCL,XEL,EBAY,ZBH,AMGN,DIS,NWL,HD,PCAR,CZR,TGT,SJM,BKNG,TWTR,BKR,DOV,GPN,IPG,ABBV,NCLH,CFG,AZO,AAL,NOC,PRU,ROL,TDY,VIAC,REG,CPB,ISRG,...,PSA,STZ,DLR,FCX,AVB,CTXS,TSN,DRE,AVY,ES,MXIM,STX,PWR,RMD,CBRE,ABMD,JCI,CERN,BR,AXP,WYNN,KSU,AKAM,COP,FMC,DUK,SYF,AMT,PFG,ORLY,BWA,O,AJG,MGM,GOOGL,BRK-B,SWK,PENN,PBCT,HAL
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2017-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-01-03,0.000012,-0.000007,0.000013,0.000006,-3.496244e-07,1.095212e-06,-0.000005,6.742953e-06,7.496907e-06,0.000002,-0.000001,0.000010,1.528459e-06,1.313798e-04,1.417918e-04,7.584275e-06,0.000010,0.000023,4.340727e-07,0.000016,4.586020e-06,0.000047,1.385736e-05,8.700071e-07,1.619568e-05,0.000016,1.669986e-06,-1.382382e-05,0.000004,8.717485e-06,0.000004,-0.000022,0.000023,1.242509e-05,-1.189297e-06,1.141297e-06,2.237790e-05,-4.479682e-07,1.535185e-06,0.000018,...,1.401374e-07,0.000024,0.000012,0.000150,-0.000003,7.367445e-06,3.055488e-05,9.701128e-07,-5.599129e-07,-1.294641e-06,6.329169e-07,5.382051e-05,6.856145e-07,-4.759556e-07,1.611761e-06,-7.988867e-07,9.452470e-05,0.000049,-0.000001,0.000062,0.000021,-0.000094,0.000009,2.589041e-05,2.693479e-06,0.000002,3.826939e-05,0.000009,6.859703e-06,0.000021,7.785511e-06,1.418350e-07,2.373527e-06,0.000052,0.000274,0.000031,0.000006,0.000003,-1.937566e-06,0.000138
2017-01-04,0.000013,0.000005,0.000004,0.000009,2.557077e-05,-1.548881e-06,0.000024,1.420702e-06,4.806642e-06,0.000018,0.000004,-0.000007,1.289571e-05,6.981564e-05,1.017293e-04,6.916750e-05,0.000041,0.000018,-5.866954e-07,0.000046,-2.785835e-06,0.000035,6.331381e-05,-3.330355e-06,2.888544e-06,0.000044,2.914033e-06,5.770502e-05,0.000036,1.069808e-05,0.000003,0.000021,0.000002,2.046334e-05,1.419103e-06,1.407197e-07,6.290408e-05,6.244895e-06,4.252987e-06,-0.000051,...,9.393851e-06,0.000085,0.000064,0.000367,0.000004,1.419150e-06,1.980446e-05,1.369556e-05,1.555854e-05,1.955824e-06,6.464232e-06,-1.392306e-05,2.181321e-06,2.671162e-06,1.030229e-05,1.083486e-05,2.528907e-06,-0.000050,0.000002,0.000050,0.000065,0.000015,0.000008,1.933782e-05,7.291399e-06,-0.000002,4.430953e-05,0.000003,4.133343e-06,0.000006,1.054355e-05,1.197440e-05,7.748563e-06,-0.000001,-0.000003,0.000008,0.000006,0.000007,8.219016e-06,0.000011
2017-01-05,0.000009,0.000003,0.000001,-0.000008,7.771678e-06,-2.157839e-05,-0.000004,1.024235e-06,-1.164017e-06,0.000030,0.000000,0.000020,8.783680e-06,3.091966e-06,-3.357766e-06,4.133114e-06,-0.000050,-0.000014,-4.983405e-07,-0.000071,3.491382e-06,0.000069,3.692835e-05,-4.543217e-06,-1.973703e-06,0.000005,-2.145020e-05,1.934123e-05,-0.000002,-1.301979e-05,-0.000002,-0.000049,0.000006,-2.056303e-05,-1.610089e-06,-8.133733e-07,-9.231285e-06,7.852283e-07,-1.545690e-06,0.000012,...,1.826032e-05,-0.000953,0.000063,-0.000065,0.000007,-1.801227e-07,6.892641e-06,-1.930721e-06,-3.880761e-06,1.291023e-07,-1.027261e-05,-1.483060e-06,-3.005875e-06,-2.614043e-06,-4.531518e-06,-3.277790e-06,-2.321890e-05,-0.000007,-0.000002,-0.000027,0.000025,0.000011,0.000021,-5.781749e-06,1.523858e-06,0.000002,-5.607945e-05,-0.000006,-6.040616e-06,0.000004,-7.651340e-07,2.786817e-05,1.322006e-06,-0.000007,0.000066,-0.000021,0.000032,-0.000006,-7.635753e-06,0.000019
2017-01-06,0.000004,0.000003,0.000005,-0.000021,-1.368060e-05,-1.019053e-06,0.000003,1.586181e-05,2.444078e-06,-0.000002,0.000002,0.000132,1.677635e-07,3.231789e-04,1.415402e-04,5.144314e-06,-0.000012,0.000011,-1.043204e-07,-0.000048,-6.206020e-07,0.000077,1.123265e-05,-5.730772e-06,2.658564e-05,0.000016,1.737985e-05,7.238139e-07,-0.000003,5.591736e-06,0.000002,0.000022,0.000004,9.384293e-06,-6.663997e-07,-6.521313e-07,-5.391328e-05,7.00

## Cumulative returns

To compare performance between the ETF and Index, we're going to calculate the tracking error. Before we do that, we first need to calculate the index and ETF comulative returns. Implement `calculate_cumulative_returns` to calculate the cumulative returns over time given the returns.

In [14]:
def calculate_cumulative_returns(returns):
  """
    Calculate cumulative returns.

    Parameters
    ----------
    returns : DataFrame
        Returns for each ticker and date

    Returns
    -------
    cumulative_returns : Pandas Series
        Cumulative returns for each date
  """

  cumulative_returns = (returns.sum(axis=1)+1).cumprod()

  return cumulative_returns

In [15]:
sp_weighted_cumulative_returns = calculate_cumulative_returns(sp_weighted_returns)
sp_weighted_cumulative_returns

Date
2017-01-02    1.000000
2017-01-03    1.008208
2017-01-04    1.018718
2017-01-05    1.016941
2017-01-06    1.022585
                ...   
2020-12-24    4.250495
2020-12-28    4.299048
2020-12-29    4.295867
2020-12-30    4.337351
2020-12-31    4.369258
Length: 1010, dtype: float64

## Covariance

If we have $m$ stock series, the covariance matrix is an $m \times m$ matrix containing the covariance between each pair of stocks.  We can use [`Numpy.cov`](https://docs.scipy.org/doc/numpy/reference/generated/numpy.cov.html) to get the covariance.  We give it a 2D array in which each row is a stock series, and each column is an observation at the same period of time. For any `NaN` values, you can replace them with zeros using the [`DataFrame.fillna`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.fillna.html) function.

The covariance matrix $\mathbf{P} = 
\begin{bmatrix}
\sigma^2_{1,1} & ... & \sigma^2_{1,m} \\ 
... & ... & ...\\
\sigma_{m,1} & ... & \sigma^2_{m,m}  \\
\end{bmatrix}$

In [16]:
def get_covariance_returns(returns):
  """
    Calculate covariance matrices.

    Parameters
    ----------
    returns : DataFrame
        Returns for each ticker and date

    Returns
    -------
    returns_covariance  : 2 dimensional Ndarray
        The covariance of the returns
  """

  returns_covariance = returns.fillna(0)
  returns_covariance = np.cov(returns_covariance.T)

  return returns_covariance

In [17]:
covariance_returns = get_covariance_returns(sp_price_returns)
covariance_returns = pd.DataFrame(covariance_returns, sp_price_returns.columns, sp_price_returns.columns)
covariance_returns

,MCO,UDR,INFO,PSX,LEN,MKC,CBOE,ROP,ECL,CCL,XEL,EBAY,ZBH,AMGN,DIS,NWL,HD,PCAR,CZR,TGT,SJM,BKNG,TWTR,BKR,DOV,GPN,IPG,ABBV,NCLH,CFG,AZO,AAL,NOC,PRU,ROL,TDY,VIAC,REG,CPB,ISRG,...,PSA,STZ,DLR,FCX,AVB,CTXS,TSN,DRE,AVY,ES,MXIM,STX,PWR,RMD,CBRE,ABMD,JCI,CERN,BR,AXP,WYNN,KSU,AKAM,COP,FMC,DUK,SYF,AMT,PFG,ORLY,BWA,O,AJG,MGM,GOOGL,BRK-B,SWK,PENN,PBCT,HAL
MCO,0.000376,0.000167,0.000224,0.000215,0.000267,0.000152,0.000150,0.000206,0.000218,0.000316,0.000135,0.000163,0.000189,0.000168,0.000186,0.000232,0.000227,0.000184,0.000415,0.000125,0.000087,0.000201,0.000266,0.000257,0.000232,0.000284,0.000210,0.000161,0.000359,0.000294,0.000160,0.000263,0.000154,0.000286,0.000166,0.000230,0.000233,0.000180,0.000083,0.000250,...,0.000126,0.000181,0.000176,0.000314,0.000179,0.000123,0.000144,0.000204,0.000208,0.000155,0.000210,0.000213,0.000207,0.000207,0.000249,0.000174,0.000193,0.000154,0.000194,0.000278,0.000283,0.000238,0.000156,0.000260,0.000241,0.000148,0.000312,0.000184,0.000281,0.000178,0.000202,0.000217,0.000206,0.000338,0.000207,0.000185,0.000306,0.000367,0.000192,0.000320
UDR,0.000167,0.000332,0.000137,0.000243,0.000213,0.000106,0.000128,0.000133,0.000182,0.000372,0.000167,0.000070,0.000161,0.000108,0.000155,0.000208,0.000151,0.000118,0.000330,0.000082,0.000068,0.000189,0.000145,0.000222,0.000184,0.000226,0.000211,0.000119,0.000339,0.000269,0.000124,0.000239,0.000134,0.000262,0.000081,0.000166,0.000180,0.000329,0.000063,0.000167,...,0.000166,0.000178,0.000165,0.000193,0.000300,0.000072,0.000122,0.000225,0.000167,0.000180,0.000131,0.000144,0.000157,0.000135,0.000245,0.000078,0.000157,0.000108,0.000112,0.000255,0.000262,0.000151,0.000064,0.000202,0.000172,0.000179,0.000284,0.000186,0.000251,0.000139,0.000143,0.000276,0.000156,0.000276,0.000120,0.000144,0.000205,0.000221,0.000240,0.000255
INFO,0.000224,0.000137,0.000301,0.000191,0.000196,0.000136,0.000099,0.000180,0.000205,0.000229,0.000123,0.000116,0.000171,0.000143,0.000169,0.000211,0.000173,0.000159,0.000339,0.000108,0.000090,0.000171,0.000173,0.000234,0.000187,0.000254,0.000201,0.000124,0.000325,0.000213,0.000122,0.000247,0.000130,0.000265,0.000138,0.000168,0.000233,0.000166,0.000067,0.000222,...,0.000099,0.000171,0.000133,0.000275,0.000140,0.000099,0.000098,0.000168,0.000208,0.000149,0.000176,0.000172,0.000180,0.000189,0.000210,0.000165,0.000146,0.000128,0.000167,0.000234,0.000220,0.000205,0.000144,0.000241,0.000166,0.000131,0.000221,0.000140,0.000249,0.000132,0.000152,0.000148,0.000154,0.000272,0.000164,0.000155,0.000248,0.000239,0.000182,0.000289
PSX,0.000215,0.000243,0.000191,0.000618,0.000299,0.000086,0.000162,0.000210,0.000247,0.000646,0.000107,0.000099,0.000265,0.000137,0.000249,0.000282,0.000181,0.000216,0.000571,0.000126,0.000049,0.000296,0.000229,0.000456,0.000272,0.000287,0.000308,0.000174,0.000624,0.000449,0.000152,0.000418,0.000162,0.000423,0.000105,0.000291,0.000299,0.000350,0.000021,0.000242,...,0.000092,0.000226,0.000111,0.000413,0.000224,0.000078,0.000197,0.000175,0.000227,0.000111,0.000210,0.000201,0.000271,0.000135,0.000357,0.000142,0.000239,0.000135,0.000166,0.000390,0.000466,0.000242,0.000097,0.000500,0.000286,0.000143,0.000418,0.000139,0.000411,0.000178,0.000279,0.000259,0.000183,0.000483,0.000194,0.000219,0.000345,0.000442,0.000344,0.000579
LEN,0.000267,0.000213,0.000196,0.000299,0.000674,0.000135,0.000136,0.000196,0.000224,0.000438,0.000119,0.000130,0.000225,0.000104,0.000194,0.000284,0.000238,0.000191,0.000586,0.000126,0.000079,0.000211,0.000241,0.000286,0.000259,0.000284,0.000239,0.000154,0.000480,0.000315,0.000193,0.000361,0.000159,0.000318,0.000151,0.000244,0.000320,0.000259,0.000040,0.000243,...,0.000141,0.000211,0.000160,0.000374,0.000215,0.000063,0.000166,0.000214,0.000214,0.000133,0.000190,0.000181,0.000270,0.000130,0.000303,0.000125,0.000233,0.000127,0.000161,0.000295,0.000346,0.000255,0.000098,0.000306,0.000255,0.000146,0.000375,0.000165,0.000333,0.000197,0.000261,0.000294,0.000207,0.000449,0.000167,0.000172,0.000382,0.000543,0.000190,0.000365
...,...,...,..

## portfolio variance
We can write the portfolio variance $\sigma^2_p = \mathbf{x^T} \mathbf{P} \mathbf{x}$

The $\mathbf{x^T} \mathbf{P} \mathbf{x}$ is called the quadratic form.
We can use the cvxpy function `quad_form(x,P)` to get the quadratic form.

## Distance from index weights
We want portfolio weights that track the index closely.  So we want to minimize the distance between them.
You can get the distance between two points in an x,y plane by adding the square of the x and y distances and taking the square root.  Extending this to any number of dimensions is called the L2 norm.  So: $\sqrt{\sum_{1}^{n}(weight_i - indexWeight_i)^2}$  Can also be written as $\left \| \mathbf{x} - \mathbf{index} \right \|_2$.  There's a cvxpy function called [norm()](https://www.cvxpy.org/api_reference/cvxpy.atoms.other_atoms.html#norm)
`norm(x, p=2, axis=None)`.  The default is already set to find 'an L2 norm, so you would pass in one argument, which is the difference between your portfolio weights and the index weights.

## objective function
We want to minimize both the portfolio variance and the distance of the portfolio weights from the index weights.
We also want to choose a `scale` constant, which is $\lambda$ in the expression. 

$\mathbf{x^T} \mathbf{P} \mathbf{x} + \lambda \left \| \mathbf{x} - \mathbf{index} \right \|_2$


This lets us choose how much priority we give to minimizing the difference from the index, relative to minimizing the variance of the portfolio by choosing a higher value for `scale` ($\lambda$).

We can find the objective function using cvxpy `objective = cvx.Minimize()`.



## constraints
We can also define our constraints in a list.  For example, you'd want the weights to sum to one. So $\sum_{1}^{n}x = 1$.  You may also need to go long only, which means no shorting, so no negative weights.  So $x_i >0 $ for all $i$. you could save a variable as `[x >= 0, sum(x) == 1]`, where x was created using `cvx.Variable()`.

## optimization
So now that we have our objective function and constraints, we can solve for the values of $\mathbf{x}$.
cvxpy has the constructor `Problem(objective, constraints)`, which returns a `Problem` object.

The `Problem` object has a function solve(), which returns the minimum of the solution.  In this case, this is the minimum variance of the portfolio.

It also updates the vector $\mathbf{x}$.

We can check out the values of $x_A$ and $x_B$ that gave the minimum portfolio variance by using `x.value`

In [18]:
def get_optimal_weights(covariance_returns, index_weights, scale=2.0):
  """
    Find the optimal weights.

    Parameters
    ----------
    covariance_returns : 2 dimensional Ndarray
        The covariance of the returns
    index_weights : Pandas Series
        Index weights for all tickers at a period in time
    scale : int
        The penalty factor for weights the deviate from the index 
    Returns
    -------
    x : 1 dimensional Ndarray
        The solution for x
  """

  assert len(covariance_returns.shape) == 2
  assert len(index_weights.shape) == 1
  assert covariance_returns.shape[0] == covariance_returns.shape[1] == index_weights.shape[0]
  
  # number of stocks m is number of rows of returns, and also number of index weights
  m = len(index_weights) 
  # covariance matrix of returns
  P = covariance_returns
  # x variables (to be found with optimisation)
  x = cvx.Variable(m)
  # portfolio variance in quadratic form
  portfolio_variance = cvx.quad_form(x, P)
  # euclidean distance (L2 norm) between portfolio and index weights
  distance_to_index = cvx.norm(x - index_weights)
  # objective function
  objective = cvx.Minimize(portfolio_variance + scale*distance_to_index)
  # constraints
  constraints = [x>=0, sum(x)==1]
  # solve objective function
  problem = cvx.Problem(objective, constraints)
  result = problem.solve()
  # get weights of optimized portfolio
  x_values = x.value

  return x_values

## Optimised portfolio without rebalancing

Using the `get_optimal_weights` function, let's generate the optimal ETF weights without rebalanceing. We can do this by feeding in the covariance of the entire history of data. We also need to feed in a set of index weights. We'll go with the average weights of the index over time.

In [19]:
raw_optimal_single_rebalance_etf_weights = get_optimal_weights(covariance_returns.values, sp_weights.iloc[-1])
print(raw_optimal_single_rebalance_etf_weights)
optimal_single_rebalance_etf_weights = pd.DataFrame(
    np.tile(raw_optimal_single_rebalance_etf_weights, (len(sp_price_returns.index), 1)),
    sp_price_returns.index,
    sp_price_returns.columns)
optimal_single_rebalance_etf_weights

[1.16305629e-03 2.66949355e-04 8.46930773e-04 7.77586980e-04
 6.89551878e-04 6.40834820e-04 2.89248842e-04 8.04278246e-04
 9.19146397e-04 3.15501861e-03 6.50081340e-04 1.01899563e-03
 4.35028046e-04 2.34092189e-03 7.26950097e-03 2.14652994e-04
 3.88443219e-03 4.27632834e-04 5.53357952e-04 2.55727713e-03
 3.41616082e-04 2.23112591e-03 1.48427111e-03 3.23287321e-04
 3.95851061e-04 1.49553945e-03 2.67720368e-04 3.60894229e-03
 2.10844459e-03 3.60886217e-04 1.13370049e-03 5.17849548e-03
 8.91554474e-04 5.21886046e-04 6.46620239e-04 3.45524854e-04
 1.60734733e-03 2.20468817e-04 2.32918254e-04 1.79541891e-03
 9.89969012e-03 7.67563093e-04 2.78787731e-03 2.88898296e-04
 1.95417960e-04 9.72705477e-04 4.93534887e-04 3.57412759e-03
 2.46056265e-03 7.42315713e-04 9.44645891e-04 6.93046171e-04
 2.61874706e-04 1.32541497e-03 6.04115462e-04 7.80694249e-04
 5.64969664e-04 6.32912138e-04 9.83781966e-04 3.75025419e-03
 5.08659392e-03 8.18499163e-04 2.90786039e-04 8.76935487e-04
 7.09328419e-04 9.268364

,MCO,UDR,INFO,PSX,LEN,MKC,CBOE,ROP,ECL,CCL,XEL,EBAY,ZBH,AMGN,DIS,NWL,HD,PCAR,CZR,TGT,SJM,BKNG,TWTR,BKR,DOV,GPN,IPG,ABBV,NCLH,CFG,AZO,AAL,NOC,PRU,ROL,TDY,VIAC,REG,CPB,ISRG,...,PSA,STZ,DLR,FCX,AVB,CTXS,TSN,DRE,AVY,ES,MXIM,STX,PWR,RMD,CBRE,ABMD,JCI,CERN,BR,AXP,WYNN,KSU,AKAM,COP,FMC,DUK,SYF,AMT,PFG,ORLY,BWA,O,AJG,MGM,GOOGL,BRK-B,SWK,PENN,PBCT,HAL
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2017-01-02,0.001163,0.000267,0.000847,0.000778,0.00069,0.000641,0.000289,0.000804,0.000919,0.003155,0.00065,0.001019,0.000435,0.002341,0.00727,0.000215,0.003884,0.000428,0.000553,0.002557,0.000342,0.002231,0.001484,0.000323,0.000396,0.001496,0.000268,0.003609,0.002108,0.000361,0.001134,0.005178,0.000892,0.000522,0.000647,0.000346,0.001607,0.00022,0.000233,0.001795,...,0.00064,0.000857,0.000794,0.002426,0.000385,0.000439,0.000689,0.000307,0.000178,0.000533,0.000679,0.001208,0.000214,0.000388,0.000298,0.000421,0.000507,0.000312,0.000237,0.001188,0.001346,0.000442,0.000485,0.001685,0.00033,0.000827,0.000452,0.002104,0.000256,0.000928,0.000264,0.000787,0.00034,0.002809,0.010467,0.005561,0.000748,0.001069,0.000205,0.000741
2017-01-03,0.001163,0.000267,0.000847,0.000778,0.00069,0.000641,0.000289,0.000804,0.000919,0.003155,0.00065,0.001019,0.000435,0.002341,0.00727,0.000215,0.003884,0.000428,0.000553,0.002557,0.000342,0.002231,0.001484,0.000323,0.000396,0.001496,0.000268,0.003609,0.002108,0.000361,0.001134,0.005178,0.000892,0.000522,0.000647,0.000346,0.001607,0.00022,0.000233,0.001795,...,0.00064,0.000857,0.000794,0.002426,0.000385,0.000439,0.000689,0.000307,0.000178,0.000533,0.000679,0.001208,0.000214,0.000388,0.000298,0.000421,0.000507,0.000312,0.000237,0.001188,0.001346,0.000442,0.000485,0.001685,0.00033,0.000827,0.000452,0.002104,0.000256,0.000928,0.000264,0.000787,0.00034,0.002809,0.010467,0.005561,0.000748,0.001069,0.000205,0.000741
2017-01-04,0.001163,0.000267,0.000847,0.000778,0.00069,0.000641,0.000289,0.000804,0.000919,0.003155,0.00065,0.001019,0.000435,0.002341,0.00727,0.000215,0.003884,0.000428,0.000553,0.002557,0.000342,0.002231,0.001484,0.000323,0.000396,0.001496,0.000268,0.003609,0.002108,0.000361,0.001134,0.005178,0.000892,0.000522,0.000647,0.000346,0.001607,0.00022,0.000233,0.001795,...,0.00064,0.000857,0.000794,0.002426,0.000385,0.000439,0.000689,0.000307,0.000178,0.000533,0.000679,0.001208,0.000214,0.000388,0.000298,0.000421,0.000507,0.000312,0.000237,0.001188,0.001346,0.000442,0.000485,0.001685,0.00033,0.000827,0.000452,0.002104,0.000256,0.000928,0.000264,0.000787,0.00034,0.002809,0.010467,0.005561,0.000748,0.001069,0.000205,0.000741
2017-01-05,0.001163,0.000267,0.000847,0.000778,0.00069,0.000641,0.000289,0.000804,0.000919,0.003155,0.00065,0.001019,0.000435,0.002341,0.00727,0.000215,0.003884,0.000428,0.000553,0.002557,0.000342,0.002231,0.001484,0.000323,0.000396,0.001496,0.000268,0.003609,0.002108,0.000361,0.001134,0.005178,0.000892,0.000522,0.000647,0.000346,0.001607,0.00022,0.000233,0.001795,...,0.00064,0.000857,0.000794,0.002426,0.000385,0.000439,0.000689,0.000307,0.000178,0.000533,0.000679,0.001208,0.000214,0.000388,0.000298,0.000421,0.000507,0.000312,0.000237,0.001188,0.001346,0.000442,0.000485,0.001685,0.00033,0.000827,0.000452,0.002104,0.000256,0.000928,0.000264,0.000787,0.00034,0.002809,0.010467,0.005561,0.000748,0.001069,0.000205,0.000741
2017-01-06,0.001163,0.000267,0.000847,0.000778,0.00069,0.000641,0.000289,0.000804,0.000919,0.003155,0.00065,0.001019,0.000435,0.002341,0.00727,0.000215,0.003884,0.000428,0.000553,0.002557,0.000342,0.002231,0.001484,0.000323,0.000396,0.001496,0.000268,0.003609,0.002108,0.000361,0.001134,0.005178,0.000892,0.000522,0.000647,0.000346,0.001607,0.00022,0.000233,0.001795,...,0.00064,0.000857,0.000794,0.002426,0.000385,0.000439,0.000689,0.000307,0.000178,0.000533,0.000679,0.001208,0.000214,0.000388,0.000298,0.000421,0.000507,0.000312,0.000237,0.001188,0.001346,0.000442,0.000485,0.001685,0.00033,0.000827,0.000452,0.002104,0.000256,0.000928,0.000264,0.000787,

With our ETF weights built, let's compare it to the index. Run the next cell to calculate the ETF returns and compare it to the index returns.

In [20]:
optim_etf_returns = generate_weighted_returns(sp_price_returns, optimal_single_rebalance_etf_weights)
optim_etf_cumulative_returns = calculate_cumulative_returns(optim_etf_returns)

## Tracking error

In order to check the performance of the smart beta portfolio, we can calculate the annualized tracking error against the index. Implement `tracking_error` to return the tracking error between the ETF and benchmark.

For reference, we'll be using the following annualized tracking error function:
$$ TE = \sqrt{252} * SampleStdev(r_p - r_b) $$

Where $ r_p $ is the portfolio/ETF returns and $ r_b $ is the benchmark returns.

_Note: When calculating the sample standard deviation, the delta degrees of freedom is 1, which is the also the default value._

In [21]:
def tracking_error(benchmark_returns_by_date, etf_returns_by_date):
    """
    Calculate the tracking error.

    Parameters
    ----------
    benchmark_returns_by_date : Pandas Series
        The benchmark returns for each date
    etf_returns_by_date : Pandas Series
        The ETF returns for each date

    Returns
    -------
    tracking_error : float
        The tracking error
    """
    assert benchmark_returns_by_date.index.equals(etf_returns_by_date.index)
    
    #TODO: Implement function
    tracking_error = (etf_returns_by_date - benchmark_returns_by_date).std()*(252**0.5)

    return tracking_error

In [22]:
optim_etf_tracking_error = tracking_error(np.sum(sp_weighted_returns, 1), np.sum(optim_etf_returns, 1))
print('Optimized ETF Tracking Error: {}'.format(optim_etf_tracking_error))

Optimized ETF Tracking Error: 0.09072089599326204


## Rebalance portfolio over time

The single optimized ETF portfolio used the same weights for the entire history. This might not be the optimal weights for the entire period. Let's rebalance the portfolio over the same period instead of using the same weights. Implement `rebalance_portfolio` to rebalance a portfolio.

Reblance the portfolio every n number of days, which is given as `shift_size`. When rebalancing, you should look back a certain number of days of data in the past, denoted as `chunk_size`. Using this data, compute the optoimal weights using `get_optimal_weights` and `get_covariance_returns`.

In [23]:
def rebalance_portfolio(returns, index_weights, shift_size, chunk_size):
    """
    Get weights for each rebalancing of the portfolio.

    Parameters
    ----------
    returns : DataFrame
        Returns for each ticker and date
    index_weights : DataFrame
        Index weight for each ticker and date
    shift_size : int
        The number of days between each rebalance
    chunk_size : int
        The number of days to look in the past for rebalancing

    Returns
    -------
    all_rebalance_weights  : list of Ndarrays
        The ETF weights for each point they are rebalanced
    """
    assert returns.index.equals(index_weights.index)
    assert returns.columns.equals(index_weights.columns)
    assert shift_size > 0
    assert chunk_size >= 0
    
    #TODO: Implement function
    all_rebalance_weights = []
    
    for i in range(chunk_size, len(returns), shift_size):
        chunks = returns.iloc[i - chunk_size : i]
        cov_returns = get_covariance_returns(chunks)
        optimal_weights = get_optimal_weights(cov_returns, index_weights.iloc[i-1])
        all_rebalance_weights.append(optimal_weights)
        
    return all_rebalance_weights

In [24]:
chunk_size = 250
shift_size = 5
all_rebalance_weights = rebalance_portfolio(sp_price_returns, sp_weights, shift_size, chunk_size)
#all_rebalance_weights

## Portfolio turnover

With the portfolio rebalanced, we need to use a metric to measure the cost of rebalancing the portfolio. Implement `get_portfolio_turnover` to calculate the annual portfolio turnover.

$ AnnualizedTurnover =\frac{SumTotalTurnover}{NumberOfRebalanceEvents} * NumberofRebalanceEventsPerYear $

$ SumTotalTurnover =\sum_{t,n}{\left | x_{t,n} - x_{t+1,n} \right |} $ Where $ x_{t,n} $ are the weights at time $ t $ for equity $ n $.

$ SumTotalTurnover $ is just a different way of writing $ \sum \left | x_{t_1,n} - x_{t_2,n} \right | $

In [25]:
def get_portfolio_turnover(all_rebalance_weights, shift_size, rebalance_count, n_trading_days_in_year=252):
    """
    Calculage portfolio turnover.

    Parameters
    ----------
    all_rebalance_weights : list of Ndarrays
        The ETF weights for each point they are rebalanced
    shift_size : int
        The number of days between each rebalance
    rebalance_count : int
        Number of times the portfolio was rebalanced
    n_trading_days_in_year: int
        Number of trading days in a year

    Returns
    -------
    portfolio_turnover  : float
        The portfolio turnover
    """
    assert shift_size > 0
    assert rebalance_count > 0
    
    #TODO: Implement function
    sum_total_turnover = np.sum(np.abs(np.diff(np.flip(all_rebalance_weights, axis=0), axis=0)))
    no_rebalance_events_per_year = n_trading_days_in_year / shift_size
    portfolio_turnover = (sum_total_turnover / rebalance_count) * no_rebalance_events_per_year
    
    return portfolio_turnover

In [26]:
print(get_portfolio_turnover(all_rebalance_weights, shift_size, len(all_rebalance_weights) - 1))

17.409750475577347
